In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

In [2]:
word_vectors = Word2Vec.load("word2vec.model").wv

In [8]:
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))

In [13]:
word_vectors.similar_by_vector(model.cluster_centers_[0], topn=10, restrict_vocab=None)

[('earth', 0.2782830595970154),
 ('4100', 0.2429516464471817),
 ('fy', 0.22961512207984924),
 ('apps', 0.22355006635189056),
 ('aapl_ong', 0.22008062899112701),
 ('10_01', 0.21893925964832306),
 ('sos_metx', 0.21289603412151337),
 ('hopefully', 0.2097347378730774),
 ('msft_4hr', 0.20859090983867645),
 ('marathon', 0.208247110247612)]

In [16]:
positive_cluster_index = 1
positive_cluster_center = model.cluster_centers_[1]
#neutral_cluser_center = model.cluster_centers_[1-positive_cluster_index]
negative_cluster_center = model.cluster_centers_[0]

In [17]:
words = pd.DataFrame(list(word_vectors.key_to_index)) #not sure if this is the right attribute

In [18]:
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

In [19]:
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [25]:
words.head(10)

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
0,amc,"[-0.00017874241, 7.881006e-05, 0.0017011166, 0...",0,-1,30.354506,-30.354506
1,!,"[-0.0027475595, 0.003099785, -6.5886976e-05, -...",1,1,29.075595,29.075595
2,gme,"[0.0027107573, -0.0014857785, -0.00035611948, ...",1,1,30.799056,30.799056
3,aapl,"[-0.0017187477, -0.0022227797, -0.0025922784, ...",0,-1,30.949872,-30.949872
4,amzn,"[-0.0006480726, -0.0017558407, 0.0031490372, -...",0,-1,29.346160,-29.346160
5,i,"[-0.002396981, 0.0014109635, 0.00072113116, 0....",1,1,28.141436,28.141436
6,msft,"[0.0032451828, -0.0032601277, -0.0021664968, 0...",0,-1,30.880918,-30.880918
7,?,"[0.0004441746, 0.0021802855, 0.003328201, 0.00...",1,1,29.964274,29.964274
8,bb,"[-0.0016578555, -0.00042776822, 0.0010935458, ...",1,1,29.765712,29.765712
9,tsla,"[0.00070450467, 0.001911718, -0.00070546946, 0...",0,-1,29.708276,-29.708276


In [28]:
words[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary.csv', index=False)

In [37]:
words[words['words']== 'short']

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
88,short,"[0.0024352146, 0.0027783306, -0.0007192111, -3...",0,-1,31.163117,-31.163117
